# Comparison of performance of SP and XP

In [1]:
from benchmark_new import *
%matplotlib inline

In [2]:
def get_data(similarity, blacklist):
    helpers = 'pdb'
    mcss = '../../../mcss_sizes.pkl'
    data = pd.concat(load(version, helpers, mcss) for version in ['stats19', 'stats20'])
    data = add_correct(data, thresh = 2.0)
    data = filter_to_ubiquitous_ligands(data)
    
    mask = data.index.get_level_values('protein') != None
    for target in blacklist:
        mask *= data.index.get_level_values('protein') != target
    data = data[mask]
    data = data[similarity(data.mcss)]

    ligand = data.xs((helpers, 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                      level=('helpers', 'mode', 'n_ligs', 'features', 'alpha')).filter(regex='rmsd|correct')
    target = ligand.groupby(level=list(range(len(ligand.index.levels)-1))).mean().dropna()
    family_ligand = ligand.groupby(level=list(range(len(ligand.index.levels)-2))).mean()
    family_target = target.groupby(level=list(range(len(target.index.levels)-1))).mean()
    
    return ligand, target, family_ligand, family_target
    
def overall_scores(ligand, target, family_ligand, family_target, ligands):
    metrics = []
    for metric, df in [('Ligand', ligand.groupby('version').mean()),
                       ('Target', target.groupby('version').mean()),
                       ('Drug-target', drug_average(family_target)),
                       ('Drug-ligand', drug_average(family_ligand))]:
        df['metric'] = metric
        df['ligands'] = ligands
        df['version'] = df.index.get_level_values('version')
        df = df.set_index(['ligands', 'metric', 'version',])
        metrics += [df.filter(regex='correct')]
    return pd.concat(metrics)

def highlight_SP_best(x):
    c = 'background-color: green;'
    m = x.groupby(level=list(range(len(x.index.names)-1)))['glide_correct'].transform('max').eq(x['glide_correct'])
    m *= x.index.get_level_values('version') == 'stats19'
    df = pd.DataFrame('', index=x.index, columns=x.columns)
    df.loc[m] += c
    return df

In [3]:
metrics = []
for slabel, sfunc in  [('All', lambda x: x > -1),
                       ('<0.5', lambda x: x < 0.5),
                       ('<0.4', lambda x: x < 0.4),
                       ('>=0.5', lambda x: x >= 0.5),
                       ('>=0.4', lambda x: x >= 0.4)]:
    for blabel, blacklist in [('All', []), ('A2AR-VDR', ['A2AR', 'VDR']), ('A2AR', ['A2AR']), ('A2AR-TRPV1', ['A2AR', 'TRPV1'])]:
        label = '{} & {}'.format(slabel, blabel)
        ligand, target, family_ligand, family_target = get_data(sfunc, blacklist)
        metrics += [overall_scores(ligand, target, family_ligand, family_target, label)]

In [4]:
pd.concat(metrics).style.apply(highlight_SP_best, axis = None)

/Users/jpaggi/miniconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [5]:
df = pd.concat(metrics)
df = df[df.index.get_level_values('metric') == 'Drug-target']
df.style.apply(highlight_SP_best, axis = None)

In [6]:
df = pd.concat(metrics)
df = df[df.index.get_level_values('metric') == 'Drug-ligand']
df.style.apply(highlight_SP_best, axis = None)